# Train Model on Chopin Nocturnes

In [1]:
import tensorflow as tf

from music_generator.model import MusicModel
from music_generator.serializers.discrete_time_serializer import DiscreteTimeMidiSerializer
import music_generator.utilities.sequence_utils as sequence_utils
import music_generator.utilities.utils as utils

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0-dev20200327


# Check if CUDA and GPU are working

In [2]:
for message in utils.check_cuda_and_gpu():
    print(message)

CUDA and GPU Available.


# Create model

The default model architecture and hyperparameters are used.

In [3]:
description = 'chopin_nocturnes_transposed'

model = MusicModel(
    ckpt_dir='./training_checkpoints/{}'.format(description),
    log_dir='./logs/{}'.format(description))

model.compile()

model.summary()

Model: "music_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         45568     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1312768   
_________________________________________________________________
dropout (Dropout)            (None, None, 512)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 512)         2048      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 512)         

# Create Dataset

A folder of MIDI files are serialized into event sequences.

All sequences are then transposed over a range from one whole-step down, to one whole-step up. This effectively multiplies the number of sequences by five, and similarly increases training time per epoch by about a factor of five. The result is reduced overfitting and better generated sequences. Whether the benefit outweighs the additional training time will depend on the training data and compositional style.

Finally, the sequences are windowed and the last event split off to use as a label for the model to train on - predicting the next event from the given sequence.

A training and validation dataset are created from the sequences and labels - the validation dataset is used in a callback to reduce the learning rate when validation loss plateaus.

In [4]:
data_path = './training_data/chopin_nocturnes/'

serializer = DiscreteTimeMidiSerializer()

sequences = serializer.serialize_folder(data_path)
sequences = sequence_utils.transpose(sequences, down=-2, up=2)
sequences, labels = sequence_utils.window(sequences)

dataset_train, dataset_val = sequence_utils.make_tf_datasets(sequences, labels)

Training Sequences: 615714
Validation Sequences: 32406


# Train Model on Dataset

Progress can also be monitored via TensorBoard.

The learning rate of the optimizer will be reduced when validation loss stalls.

The checkpoint with the best training loss will be saved.

In [ ]:
epochs = 100

try:
    history = model.fit(dataset_train,
                    validation_data=dataset_val,
                    epochs=epochs,
                    verbose=0)
except KeyboardInterrupt as e:
    print('\nStopping training...')

# Sample from model

Load the checkpoint from training with the lowest training loss and generate three midi files using a single chord for a seed.

The resulting sequence generated is heavily influenced by the chosen seed - some experimentation is required here to find good seed choices for a particular compositional style.

In [5]:
ckpt_dir = './training_checkpoints/{}'.format(description)
model.load_checkpoint(ckpt_dir, use_latest=True)

# Generate MIDI files using different seeds
# These below represent a B-minor, G7, and A-minor chord, as well as several individual notes
seeds = [[38, 47, 54, 62, 66],
         [43, 55, 59, 62, 65],
         [45, 60, 64, 69],
         [79],
         [95],
         [42]]

length = 1000

for seed in seeds:
    seed_string = '-'.join([str(x) for x in seed])
    generated_sequence = model.generate_sequence(length, seed)
    serializer.deserialize(generated_sequence, './generated_files/{}'.format(description), 'sample_{}.mid'.format(seed_string))